In [7]:
# Install packages
%pip install --upgrade python-dotenv langchain chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 8.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 27.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.52
    Uninstalling langchain-core-0.1.52:
      Successfully uninstalled langchain-core-0.1.52
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.0.2
    Uninstalling langchain-text-splitters-0.0.2:
      Successfully uninstalled langchain-text-splitters-0.0.2
  Attempting uninstall: langchain
    Found existing installation: langchain 0.1.20
    Uninstalling langchain-0.1.20:
      Successfully uninstalled langchain-0.1.20
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.0.38 requires langchain-core<0.2.0,>=0.1.52, but you h

In [9]:
import logging
import sys

# Turn off unwanted logs
logging.basicConfig(stream=sys.stdout, level=logging.ERROR)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [10]:
%pip install openai 
%pip install tiktoken

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [45]:
from langchain.document_loaders import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

loader = CSVLoader('./data/products_data.csv', csv_args={})
documents = loader.load()

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(documents, embeddings, persist_directory="./knownledge/")
docsearch.persist()

In [112]:
%pip install -U langchain-openai


Note: you may need to restart the kernel to use updated packages.


In [46]:
from dotenv import load_dotenv

load_dotenv()

True

In [47]:
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(temperature=0)

In [48]:
import openai
import pandas as pd
from langchain.document_loaders import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.agents import initialize_agent, Tool
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain import VectorDBQA
import os
import re

docsearch = Chroma(embedding_function=embeddings, persist_directory="./knownledge/")

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")


system_prompt = PromptTemplate(
    template="""
You are a shopping assistant and your name is EKKA AI that helps visitors find the right product from the store.
If you can't find the answer from the context, ask more questions. Do not make up products.
Once you have an answer, list down the products in a clear tone as a markdown list.
You also provide additional shopping support, including comparing products and making recommendations based on user preferences.
Here are some rules you must follow:
Here are some rules you must follow:
1. Always provide accurate and relevant information.
2. If a product cannot be found, politely ask for more details.
3. When comparing products, use criteria such as price, rating, and features.
4. When making recommendations, consider the user's preferences and criteria provided.
5. Do not fabricate information or products. Stick to the data provided.
6. Ask the user if they need a comparison. If the user provides product links, find and compare them. Provide the best product.
----------------
{context}""",
    input_variables=["context"],
)

system_message_prompt = SystemMessagePromptTemplate(prompt=system_prompt)
human_message_prompt = HumanMessagePromptTemplate.from_template("{question}")

chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt],
)

# Định nghĩa công cụ tìm kiếm sản phẩm
product_search = VectorDBQA.from_chain_type(
    llm=chat, chain_type="stuff", vectorstore=docsearch
)

/opt/anaconda3/lib/python3.11/site-packages/langchain/chains/retrieval_qa/base.py:289: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


In [49]:
from langchain import VectorDBQA
from langchain.agents import Tool

product_search = VectorDBQA.from_chain_type(
    llm=chat, chain_type="stuff", vectorstore=docsearch
)

tools = [
    Tool(
        name="Product Search",
        func=product_search.run,
         description="useful for when you need to find products available in the store. If the user is looking for comparisons, ask if they need a product comparison, and if so, compare and provide the best product."
    ),
]

/opt/anaconda3/lib/python3.11/site-packages/langchain/chains/retrieval_qa/base.py:289: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


In [50]:
from langchain.agents import initialize_agent
from langchain.memory import ConversationBufferMemory
# Define memory and prefix

# Define memory and prefix
prefix_default = """
Assistant is a large language model trained by HoangDieuIT.
Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. 
As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.
Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. 
Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.
Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.
"""

prefix = """
You are an AI Shopping Assistant and the shop name is EKKA, designed to be able to assist the user in finding the right product in an online store. The store contains products
for various categories such as clothing, electronics, jewelry, and more.
You are given the following filtered products in a shop and a conversation. You should try better to understand the user's needs and suggest one or more products. 
Provide a conversational answer based on the products provided. If you have more than one product to recommend, show them as a bulleted list.
If you can't find the answer in the context below, say politely "Hmm, I'm not sure." Don't try to make up a product which is not.
"""

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

openai.api_key = openai_api_key
chat = ChatOpenAI(api_key=openai_api_key, model="gpt-3.5-turbo-0125")

ask_ai = initialize_agent(
    tools=tools,
    llm=chat,
    agent="chat-conversational-react-description",
    verbose=True,
    memory=memory,
    prefix=prefix,
    input_variables=["input", "agent_scratchpad"],
)


In [51]:
ask_ai.run("give me 10 socks links")



> Entering new AgentExecutor chain...
```json
{
    "action": "Product Search",
    "action_input": "10 socks links"
}
```
Observation: Here are 10 links to different socks products:

1. [Happy Socks Unisex Multi Color Socks](http://localhost:8000/store/accessories/socks/happy-socks-unisex-multi-color-socks-37891)
2. [Happy Socks Unisex Multi Color Socks](http://localhost:8000/store/accessories/socks/happy-socks-unisex-multi-color-socks-37883)
3. [Nike Men Pack of 3 Sports Socks](http://localhost:8000/store/accessories/socks/nike-men-pack-of-3-sports-socks-15275)
4. [Happy Socks Unisex Multi Color Socks](http://localhost:8000/store/accessories/socks/happy-socks-unisex-multi-color-socks-37889)
Thought:```json
{
    "action": "Final Answer",
    "action_input": "Here are 10 links to different socks products: 1. Happy Socks Unisex Multi Color Socks 2. Happy Socks Unisex Multi Color Socks 3. Nike Men Pack of 3 Sports Socks 4. Happy Socks Unisex Multi Color Socks"
}
```

> Finished chain.

'Here are 10 links to different socks products: 1. Happy Socks Unisex Multi Color Socks 2. Happy Socks Unisex Multi Color Socks 3. Nike Men Pack of 3 Sports Socks 4. Happy Socks Unisex Multi Color Socks'

In [38]:
ask_ai.run("socks white color")



> Entering new AgentExecutor chain...
```json
{
    "action": "Product Search",
    "action_input": "white color socks"
}
```
Observation: Here are some options for white color socks:

1. United Colors of Benetton Men Pack of 2 White Socks
   - Price: $1.22
   - Brand: United Colors of Benetton
   - Gender: Men
   - Season: Summer
   - Year: 2011
   - [Product Link](http://localhost:8000/store/accessories/socks/united-colors-of-benetton-men-pack-of-2-white-socks-14600)

2. Adidas Unisex White Socks
   - Price: $1.99
   - Brand: Adidas
   - Gender: Unisex
   - Season: Summer
   - Year: 2012
   - [Product Link](http://localhost:8000/store/accessories/socks/adidas-unisex-white-socks-32472)

3. Nike Women Sports White Socks
   - Price: $1.25
   - Brand: Nike
   - Gender: Women
   - Season: Fall
   - Year: 2011
   - [Product Link](http://localhost:8000/store/accessories/socks/nike-women-sports-white-socks-12737)
Thought:```json
{
    "action": "Final Answer",
    "action_input": "Here are

'Here are some options for white color socks: 1. United Colors of Benetton Men Pack of 2 White Socks - Price: $1.22 - Brand: United Colors of Benetton - Gender: Men - Season: Summer - Year: 2011 2. Adidas Unisex White Socks - Price: $1.99 - Brand: Adidas - Gender: Unisex - Season: Summer - Year: 2012 3. Nike Women Sports White Socks - Price: $1.25 - Brand: Nike - Gender: Women - Season: Fall - Year: 2011'

In [39]:
ask_ai.run("compare them and give me best socks")



> Entering new AgentExecutor chain...
```json
{
    "action": "Product Search",
    "action_input": "white color socks"
}
```
Observation: Here are some white color socks available in the store:

1. United Colors of Benetton Men Stripes White Socks
2. Adidas Unisex White Socks
3. United Colors of Benetton Men Pack of 2 White Socks
4. Nike Women Sports White Socks

You can find more details about each product by clicking on the provided links.
Thought:```json
{
    "action": "Final Answer",
    "action_input": "Here are some white color socks available: 1. United Colors of Benetton Men Stripes White Socks 2. Adidas Unisex White Socks 3. United Colors of Benetton Men Pack of 2 White Socks 4. Nike Women Sports White Socks"
}
```

> Finished chain.


'Here are some white color socks available: 1. United Colors of Benetton Men Stripes White Socks 2. Adidas Unisex White Socks 3. United Colors of Benetton Men Pack of 2 White Socks 4. Nike Women Sports White Socks'

In [42]:
ask_ai.run("give me the best")



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "I'm here to assist you with finding the best option. How can I help you further?"
}
```

> Finished chain.


"I'm here to assist you with finding the best option. How can I help you further?"

In [43]:
ask_ai.run("give me some men watch that are cheap")



> Entering new AgentExecutor chain...
```json
{
    "action": "Product Search",
    "action_input": "men's watches cheap"
}
```
Observation: Here are a few men's watches with affordable prices:

1. [Citizen Men Black Dial Watch](http://localhost:8000/store/accessories/watches/citizen-men-black-dial-watch-33895) - Price: $58.00
2. [Citizen Men Gold Dial Watch](http://localhost:8000/store/accessories/watches/citizen-men-gold-dial-watch-33913) - Price: $59.00
3. [Q&Q Men Black Dial Watch](http://localhost:8000/store/accessories/watches/qq-men-black-dial-watch-38008) - Price: $13.95

These watches offer good value for their prices.
Thought:```json
{
    "action": "Final Answer",
    "action_input": "Here are a few men's watches with affordable prices: 1. Citizen Men Black Dial Watch - Price: $58.00 2. Citizen Men Gold Dial Watch - Price: $59.00 3. Q&Q Men Black Dial Watch - Price: $13.95 These watches offer good value for their prices."
}
```

> Finished chain.


"Here are a few men's watches with affordable prices: 1. Citizen Men Black Dial Watch - Price: $58.00 2. Citizen Men Gold Dial Watch - Price: $59.00 3. Q&Q Men Black Dial Watch - Price: $13.95 These watches offer good value for their prices."

In [44]:
ask_ai.run("compare Citizen Men Black Dial Watch and Citizen Men Gold Dial Watch")



> Entering new AgentExecutor chain...
```json
{
    "action": "Product Search",
    "action_input": "Citizen Men Black Dial Watch"
}
```
Observation: What would you like to know about the Citizen Men Black Dial Watch?
Thought:```json
{
    "action": "Final Answer",
    "action_input": "Citizen Men Black Dial Watch is priced at $58.00."
}
```

> Finished chain.


'Citizen Men Black Dial Watch is priced at $58.00.'

In [32]:
ask_ai.run("give me urls")



> Entering new AgentExecutor chain...
```json
{
    "action": "Product Search",
    "action_input": "white watches Citizen"
}
```
Observation: The Citizen watches in white for women are available in different styles and specifications. Here are a few options:

1. Citizen Women White Dial Watch - Case diameter: 18 mm, Case thickness: 3 mm, Dial shape: Rectangle, Stainless steel rectangular case with crystal embellishments, Steel strap with a butterfly clasp, White dial with metallic hands, Water resistant. Price: $55.00. [Product URL](http://localhost:8000/store/accessories/watches/citizen-women-white-dial-watch-33916)

2. Citizen Women White Dial Watch - Case diameter: 21 mm, Case thickness: 5 mm, Dial shape: Round, Stainless steel round case with a golden bezel, Steel strap with a butterfly clasp, White dial with golden hands, Water resistance up to 50 meters. Price: $75.00. [Product URL](http://localhost:8000/store/accessories/watches/citizen-women-white-dial-watch-33929)

3. Citiz

'I recommend the Citizen Women White Dial Watch with a round case, golden bezel, steel strap, butterfly clasp, white dial, and golden hands priced at $75.00. You can find more details and purchase it [here](http://localhost:8000/store/accessories/watches/citizen-women-white-dial-watch-33929).'

In [33]:
ask_ai.run("do you know how to buy it?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "You can purchase the Citizen Women White Dial Watch by clicking on the following link: [Purchase Watch](http://localhost:8000/store/accessories/watches/citizen-women-white-dial-watch-33929)"
}
```

> Finished chain.


'You can purchase the Citizen Women White Dial Watch by clicking on the following link: [Purchase Watch](http://localhost:8000/store/accessories/watches/citizen-women-white-dial-watch-33929)'

In [ ]:
import IPython

while True:
    request = input("Input: ")
    IPython.display.display(IPython.display.Markdown("User: " + request))

    if request == "Bye" or request == "bye":
        IPython.display.display(IPython.display.Markdown("Bot: Bye!"))
        break
    else:
        response = ask_ai.run(request)
        IPython.display.display(IPython.display.Markdown("Bot: " + response))


In [ ]:
memory.buffer